In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import pandas as pd
import torch
from random import randint

from fairseq_signals.utils.store import MemmapReader

In [3]:
root = '/home/aa2650/datasets/code_15/subset'
experiment_root = '/home/aa2650/datasets/code_15/experiments/subset'
fairseq_signals_root = '/home/aa2650/playground/fairseq-signals'
fairseq_signals_root = fairseq_signals_root.rstrip('/')
fairseq_signals_root

data_split = "80-10-10"

The segmented split must be saved with absolute file paths, so we will update the current relative file paths accordingly.

In [4]:
# The bottom part only needs to be run once
segmented_path = f'/home/aa2650/datasets/code_15/subset/test_segmented_split.csv'
segmented_split = pd.read_csv(segmented_path,
    index_col='idx',
)
# segmented_split['path'] = ('/home/aa2650/datasets/code_15/segmented/') + segmented_split['path']
# segmented_split.to_csv(os.path.join('/home/aa2650/datasets/code_15/segmented_split.csv'))

Run the follow commands togenerate the `test.tsv` file used for inference.

In [ ]:
run = "3"

In [149]:
each_experiment_path = os.path.join(experiment_root, data_split, "100", run)
os.makedirs(each_experiment_path, exist_ok=True)

model_path = f"/home/aa2650/playground/ECG-FM/experiments/subset/5/checkpoint8.pt"

print(each_experiment_path)


/home/aa2650/datasets/code_15/experiments/subset/80-10-10/100/2


In [150]:
inference_cmd = f"""fairseq-hydra-inference \\
    task.data="/home/aa2650/datasets/code_15/subset/manifests" \\
    common_eval.path="{model_path}" \\
    common_eval.results_path="{each_experiment_path}" \\
    model.num_labels=6 \\
    dataset.valid_subset="test" \\
    dataset.batch_size=10 \\
    dataset.num_workers=3 \\
    dataset.disable_validation=false \\
    distributed_training.distributed_world_size=1 \\
    distributed_training.find_unused_parameters=True \\
    --config-dir "/home/aa2650/playground/ECG-FM/ckpts/" \\
    --config-name physionet_finetuned
"""

os.system(inference_cmd)

/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


[2025-05-16 14:09:57,633][fairseq_cli.inference][INFO] - loading model from /home/aa2650/playground/ECG-FM/experiments/subset/5/checkpoint8.pt
[2025-05-16 14:09:59,286][fairseq_signals.utils.checkpoint_utils][INFO] - Loaded a checkpoint in 1.65s
[2025-05-16 14:09:59,287][fairseq_cli.inference][INFO] - num. shared model params: 90,377,862 (num. trained: 90,377,862)
[2025-05-16 14:09:59,289][fairseq_cli.inference][INFO] - num. expert model params: 0 (num. trained: 0)
[2025-05-16 14:09:59,448][fairseq_cli.inference][INFO] - {'_name': None,
 'checkpoint': {'_name': None, 'save_dir': '/home/aa2650/playground/ECG-FM/experiments/subset/5', 'restore_file': 'checkpoint_last.pt', 'finetune_from_model': None, 'reset_dataloader': False, 'reset_lr_scheduler': False, 'reset_meters': False, 'reset_optimizer': False, 'optimizer_overrides': '{}', 'save_interval': 1, 'save_interval_updates': 0, 'keep_interval_updates': -1, 'keep_interval_updates_pattern': -1, 'keep_last_epochs': 0, 'keep_best_checkpoint

0

In [151]:
assert os.path.isfile(f"{each_experiment_path}/outputs_test.npy")
assert os.path.isfile(f"{each_experiment_path}/outputs_test_header.pkl")

## 4. Interpret results

The logits are ordered same as the samples in the manifest and labels in the label definition.

### Get predictions on PhysioNet 2021 labels

In [152]:
# os.path.join('/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv'),

code15_label_def = pd.read_csv(
    os.path.join('/home/aa2650/datasets/code_15/subset/label_def.csv'),
     index_col='name',
)
code15_label_names = code15_label_def.index
code15_label_names

Index(['RBBB', 'LBBB', 'SB', 'ST', 'AF', 'normal_ecg'], dtype='object', name='name')

In [153]:
# Load the array of computed logits
logits = MemmapReader.from_header(f"{each_experiment_path}/outputs_test.npy")[:]
logits.shape

(5873, 6)

In [154]:
# Construct predictions from logits
pred = pd.DataFrame(
    torch.sigmoid(torch.tensor(logits)).numpy(),
    columns=code15_label_names,
)

pred = segmented_split.reset_index().join(pred, how='left').set_index('idx')
pred

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg
idx,,,,,,,,,,
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,3.911330e-04,9.995859e-01,4.313099e-05,0.001211,3.301499e-03,0.000004
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.515775e-03,9.993563e-01,1.332293e-04,0.000740,6.556175e-03,0.000007
29077,code_15_100123.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,2.177472e-05,9.999775e-01,5.610372e-06,0.000163,3.284064e-04,0.000015
12710,code_15_1001938.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,4.546802e-07,1.258478e-07,4.609850e-06,0.999938,9.342390e-07,0.000001
35541,code_15_1002557.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,2.267948e-04,5.203434e-04,1.081094e-05,0.000464,3.351842e-04,0.999854
...,...,...,...,...,...,...,...,...,...,...
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.983702e-01,8.499264e-04,8.665058e-06,0.000560,8.912811e-03,0.000013
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.989479e-01,6.531616e-04,3.343615e-06,0.000257,8.615340e-03,0.000004
37132,code_15_998911.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,6.904961e-04,6.434451e-05,1.286032e-06,0.000069,3.058049e-04,0.999977


In [155]:
# Perform a (crude) thresholding of 0.5 for all labels
pred_thresh = pred.copy()
pred_thresh[code15_label_names] = pred_thresh[code15_label_names] > 0.5

# Construct a readable column of predicted labels for each sample
pred_thresh['labels'] = pred_thresh[code15_label_names].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
pred_thresh['labels']

idx
4329           LBBB
4329           LBBB
29077          LBBB
12710            ST
35541    normal_ecg
            ...    
15687          RBBB
15687          RBBB
37132    normal_ecg
418            RBBB
37915    normal_ecg
Name: labels, Length: 5873, dtype: object

In [156]:
code_15_label_def = pd.read_csv("/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv",
     index_col='name',
)
code_15_label_names = code_15_label_def.index
code_15_label_def

,pos_count_all,pos_percent_all
name,,
is_male,138528,0.402691
1dAVb,5699,0.016567
RBBB,9652,0.028058
LBBB,6011,0.017474
SB,5588,0.016244
ST,7571,0.022008
AF,7008,0.020372
normal_ecg,134497,0.390973


In [157]:
label_mapping = {
    'RBBB': 'RBBB',
    'LBBB': 'LBBB',
    'SB': 'SB',
    'ST': 'ST',
    'AF': 'AF',
    'normal_ecg': 'normal_ecg'
}

code15_label_def['name_mapped'] = code15_label_def.index.map(label_mapping)
code15_label_def

,pos_count_all,pos_percent_all,name_mapped
name,,,
RBBB,9672,0.224518,RBBB
LBBB,6026,0.139883,LBBB
SB,5605,0.130110,SB
ST,7584,0.176049,ST
AF,7033,0.163258,AF
normal_ecg,9500,0.220525,normal_ecg


In [158]:
pred_mapped = pred.copy()
pred_mapped.drop(set(code15_label_names) - set(label_mapping.keys()), axis=1, inplace=True)
pred_mapped.rename(label_mapping, axis=1, inplace=True)
pred_mapped

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg
idx,,,,,,,,,,
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,3.911330e-04,9.995859e-01,4.313099e-05,0.001211,3.301499e-03,0.000004
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.515775e-03,9.993563e-01,1.332293e-04,0.000740,6.556175e-03,0.000007
29077,code_15_100123.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,2.177472e-05,9.999775e-01,5.610372e-06,0.000163,3.284064e-04,0.000015
12710,code_15_1001938.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,4.546802e-07,1.258478e-07,4.609850e-06,0.999938,9.342390e-07,0.000001
35541,code_15_1002557.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,2.267948e-04,5.203434e-04,1.081094e-05,0.000464,3.351842e-04,0.999854
...,...,...,...,...,...,...,...,...,...,...
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.983702e-01,8.499264e-04,8.665058e-06,0.000560,8.912811e-03,0.000013
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.989479e-01,6.531616e-04,3.343615e-06,0.000257,8.615340e-03,0.000004
37132,code_15_998911.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,6.904961e-04,6.434451e-05,1.286032e-06,0.000069,3.058049e-04,0.999977


In [159]:
pred_thresh_mapped = pred_thresh.copy()
pred_thresh_mapped.drop(set(code15_label_names) - set(label_mapping.keys()), axis=1, inplace=True)
pred_thresh_mapped.rename(label_mapping, axis=1, inplace=True)
pred_thresh_mapped['predicted'] = pred_thresh_mapped[label_mapping.values()].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
pred_thresh_mapped

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg,labels,predicted
idx,,,,,,,,,,,,
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,False,False,False,False,LBBB,LBBB
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,False,False,False,False,LBBB,LBBB
29077,code_15_100123.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,False,False,False,False,LBBB,LBBB
12710,code_15_1001938.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,True,False,False,ST,ST
35541,code_15_1002557.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,False,True,normal_ecg,normal_ecg
...,...,...,...,...,...,...,...,...,...,...,...,...
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,True,False,False,False,False,False,RBBB,RBBB
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,True,False,False,False,False,False,RBBB,RBBB
37132,code_15_998911.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,False,True,normal_ecg,normal_ecg


In [160]:
true_labels = pd.read_csv(os.path.join('/home/aa2650/datasets/code_15/subset/ground_truth_test_labels.csv'), index_col='idx')
true_labels['actual'] = true_labels[label_mapping.values()].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
true_labels

,RBBB,LBBB,SB,ST,AF,normal_ecg,actual
idx,,,,,,,
37252,False,False,False,False,False,True,normal_ecg
37252,False,False,False,False,False,True,normal_ecg
22756,False,True,False,False,False,False,LBBB
22756,False,True,False,False,False,False,LBBB
2342,True,False,False,False,True,False,"RBBB, AF"
...,...,...,...,...,...,...,...
418,True,False,False,False,False,False,RBBB
12704,True,False,False,False,False,False,RBBB
32348,False,True,False,False,False,False,LBBB


In [161]:
# Visualize predicted and actual labels side-by-side
pred_thresh_mapped[['predicted']].join(true_labels[['actual']], how='left')

# Calculate accuracy
# Compare predicted and actual labels
comparison = pred_thresh_mapped[['predicted']].join(true_labels[['actual']], how='left')

# Calculate overall accuracy (exact match)
accuracy = (comparison['predicted'] == comparison['actual']).mean()
print(f"Overall accuracy: {accuracy:.2%}")

Overall accuracy: 92.05%


In [162]:
from imblearn.metrics import specificity_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score
)

y_true = comparison['actual']
y_pred = comparison['predicted']

accuracy  = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall    = recall_score(y_true, y_pred, average='weighted')
f1        = f1_score(y_true, y_pred, average='weighted')
specificity = specificity_score(y_true, y_pred, average='weighted')

print(f"Accuracy:    {accuracy:.3f}")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"F1 score:    {f1:.3f}")


Accuracy:    0.921
Precision:   0.930
Recall:      0.921
Specificity: 0.990
F1 score:    0.925


/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [163]:
y_true_str = comparison['actual']
y_pred_str = comparison['predicted']

y_true_list = [labels.split(", ") for labels in y_true_str]
y_pred_list = [labels.split(", ") for labels in y_pred_str]

mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(y_true_list)
y_pred_bin = mlb.transform(y_pred_list)

class_names = mlb.classes_

results = []

for i, cls_name in enumerate(class_names):
    y_true_col = y_true_bin[:, i]
    y_pred_col = y_pred_bin[:, i]
    
    tn, fp, fn, tp = confusion_matrix(y_true_col, y_pred_col).ravel()
    total = tp + tn + fp + fn
    prevalence = (tp + fn) / total
    sensitivity = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
    specificity = tn / (tn + fp)
    npv = tn / (tn + fn)
    accuracy = (tp + tn) / total

    results.append({
        'class': cls_name,
        'prevalence': round(prevalence, 3),
        'f1': round(f1, 3),
        'precision': round(precision, 3),
        'recall': round(sensitivity, 3),
        'specificity': round(specificity, 3),
        'accuracy': round(accuracy, 3)
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)

        class  prevalence     f1  precision  recall  specificity  accuracy
0          AF       0.157  0.933      0.948   0.919        0.991     0.979
1        LBBB       0.151  0.956      0.965   0.948        0.994     0.987
2        RBBB       0.202  0.971      0.973   0.968        0.993     0.988
3          SB       0.134  0.914      0.939   0.891        0.991     0.978
4          ST       0.176  0.943      0.931   0.956        0.985     0.980
5  normal_ecg       0.231  0.952      0.933   0.973        0.979     0.977


/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


In [164]:
with open(f"/home/aa2650/playground/ECG-FM/results/finetuned/finetuned-results-{run}.txt", "w") as f:
    f.write(metrics_df.drop(columns=['prevalence']).to_string(index=False))